<a href="https://colab.research.google.com/github/kimdonggyu2008/hufs_hackerthon/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%95%A9%EC%B9%98%EA%B3%A0_%EC%A6%9D%EA%B0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from nltk.corpus import wordnet
from sklearn.utils import resample

# nltk 데이터 다운로드 (한 번만 실행 필요)
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import pandas as pd
import random
from sklearn.utils import resample

# 데이터 로드
train = pd.read_csv('/content/drive/MyDrive/코딩공부/dacon_project/data/train.csv')

def shuffle_sentences(text):
    sentences = text.split('.')  # 문장을 마침표로 분리
    random.shuffle(sentences)     # 문장 순서를 무작위로 섞기
    return '.'.join(sentences)

# 기사와 키워드 2개씩 합치기
def combine_articles(df):
    combined_data = []

    for category in df['분류'].unique():
        category_data = df[df['분류'] == category].reset_index(drop=True)

        # 5개씩 묶어서 합치기
        for i in range(0, len(category_data), 2):
            combined_title = " ".join(category_data['제목'][i:i+2])
            combined_keywords = ",".join(category_data['키워드'][i:i+2])
            combined_entry = {
                '분류': category,
                '제목': combined_title,
                '키워드': combined_keywords
            }
            combined_data.append(combined_entry)

    return pd.DataFrame(combined_data)

# 카테고리별로 5개씩 합친 데이터프레임 생성
combined_df = combine_articles(train)

#데이터 증강
def augment_data(df, target_count=200):
    augmented_df = df.copy()

    # 각 카테고리별로 데이터 증강
    for category in df['분류'].unique():
        category_data = augmented_df[augmented_df['분류'] == category]
        current_count = len(category_data)

        # 200개 미만이면 증강
        if current_count < target_count:
            additional_samples_needed = target_count - current_count
            augmented_entries = resample(category_data, replace=True, n_samples=additional_samples_needed, random_state=42)

            augmented_entries['제목'] = augmented_entries['제목'].apply(shuffle_sentences)
            augmented_entries['키워드'] = augmented_entries['키워드'].apply(shuffle_sentences)

            augmented_df = pd.concat([augmented_df, augmented_entries], ignore_index=True)

    return augmented_df


combined_df = combine_articles(train)

# 각 카테고리 당 최소 50개로 증강 (문장 순서 변경 포함)
augmented_df = augment_data(combined_df, target_count=200)

# 카테고리별로 10000개가 넘는 경우 5000개로 줄이기
def reduce_large_categories(df, max_count=7000, threshold=10000):
    reduced_df = pd.DataFrame()

    for category in df['분류'].unique():
        category_data = df[df['분류'] == category]
        if len(category_data) > threshold:
            # 10000개가 넘는 경우, 랜덤으로 5000개 선택
            category_data = resample(category_data, replace=False, n_samples=max_count, random_state=42)
        reduced_df = pd.concat([reduced_df, category_data], ignore_index=True)

    return reduced_df

# 카테고리 별로 10000개가 넘는 경우 5000개로 줄이기
final_df = reduce_large_categories(augmented_df, max_count=10000, threshold=10000)

# 줄인 데이터 확인
print(augmented_df['분류'].value_counts())
# 증강된 데이터 저장 (선택 사항)
augmented_df.to_csv('/content/drive/MyDrive/코딩공부/dacon_project/data/train_augmented_combined6.csv', index=False)
